<a href="https://colab.research.google.com/github/mok3bat/DataDevQuest/blob/main/DDQ2025_05_Multi_Source_Excel_data_to_Tableau_Hyper_with_SQL_Queries_(Intermediate).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Libraries
! pip install pandas pantab tableauhyperapi ipython-autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.6 MB/s eta 0:00:00


In [212]:
# import Libraries

'''
The selected code cell imports several Python libraries and modules that are commonly used for data manipulation, particularly when working with large datasets and Tableau Hyper files.

Here's a breakdown of the imports:

itertools: This module implements a number of iterator building blocks inspired by constructs from APL, Haskell, and SML. It provides tools for working with iterators efficiently.
time: This module provides various time-related functions, often used for measuring the execution time of code.
numpy as np: Imports the NumPy library, aliasing it as np. NumPy is fundamental for numerical operations in Python, especially for working with arrays and matrices.
pandas as pd: Imports the pandas library, aliasing it as pd. Pandas is a powerful library for data manipulation and analysis, providing data structures like DataFrames.
pantab as pt: Imports the pantab library, aliasing it as pt. Pantab is a library that allows you to read and write pandas DataFrames to and from Tableau Hyper files.
from tableauhyperapi import ...: This line imports specific components from the tableauhyperapi library. This library is the official Python API for working directly with Tableau Hyper files. The imported components are:
Connection: Used to establish a connection to a Hyper file.
CreateMode: Defines how to handle the creation of a Hyper file (e.g., create if it doesn't exist, replace if it does).
HyperProcess: Manages the Hyper process, which is the engine that interacts with Hyper files.
Inserter: Used to insert data into a table within a Hyper file.
SqlType: Represents SQL data types used in Hyper tables.
TableDefinition: Defines the schema (table name, column names, data types) of a table in a Hyper file.
TableName: Represents the name of a table in a Hyper file.
Telemetry: Used to configure telemetry settings for the Hyper process.
'''

import itertools
import time

import numpy as np
import pandas as pd
import pantab as pt
from tableauhyperapi import (
    Connection,
    CreateMode,
    HyperProcess,
    Inserter,
    SqlType,
    TableDefinition,
    TableName,
    Telemetry,
    escape_name,
    escape_string_literal
)

from tabulate import tabulate
import datetime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.46 ms (started: 2025-06-10 10:59:19 +00:00)


In [3]:
excel_path_1 = "https://github.com/nikdutra/DDQ-2025-04/raw/refs/heads/main/datasets/RWFD_Supply_Chain.xlsx"
sheet_name_1 = "OrderList"

df_orders = pd.read_excel(excel_path_1, sheet_name=sheet_name_1)

print(tabulate(df_orders.head(), headers='keys', tablefmt='psql'))

+----+-------------+---------------------+---------------+-----------+-------+-----------------+------------------------+-----------------------+------------+--------------+--------------+--------------------+-----------------+----------+
|    |    Order ID | Order Date          | Origin Port   | Carrier   |   TPT | Service Level   |   Ship ahead day count |   Ship Late Day count | Customer   |   Product ID | Plant Code   | Destination Port   |   Unit quantity |   Weight |
|----+-------------+---------------------+---------------+-----------+-------+-----------------+------------------------+-----------------------+------------+--------------+--------------+--------------------+-----------------+----------|
|  0 | 1.4473e+09  | 2013-05-26 00:00:00 | PORT09        | V44_3     |     1 | CRF             |                      3 |                     0 | V55555_53  |      1700106 | PLANT16      | PORT09             |             808 |    14.3  |
|  1 | 1.44716e+09 | 2013-05-26 00:00:00 | P

In [18]:
# Check the inferred data types
print(df_orders.dtypes)

Order ID                         int64
Order Date              datetime64[ns]
Origin Port                     object
Carrier                         object
TPT                              int64
Service Level                   object
Ship ahead day count             int64
Ship Late Day count              int64
Customer                        object
Product ID                       int64
Plant Code                      object
Destination Port                object
Unit quantity                    int64
Weight                         float64
dtype: object
time: 14.2 ms (started: 2025-06-10 07:40:24 +00:00)


**All types looks fine for me. Except for Order ID, Product ID which should be Int**

In [19]:
# Change column datatype
convert_dict = {'Product ID': int, 'Order ID': int}

df_orders = df_orders.astype(convert_dict)
print(df_orders.dtypes)

Order ID                         int64
Order Date              datetime64[ns]
Origin Port                     object
Carrier                         object
TPT                              int64
Service Level                   object
Ship ahead day count             int64
Ship Late Day count              int64
Customer                        object
Product ID                       int64
Plant Code                      object
Destination Port                object
Unit quantity                    int64
Weight                         float64
dtype: object
time: 5.93 ms (started: 2025-06-10 07:40:31 +00:00)


In [197]:

excel_path_2 = "https://github.com/nikdutra/DDQ-2025-04/raw/refs/heads/main/datasets/RWFD_Solar_Energy.xlsx"
sheet_name_2 = "Actuals"

df_actuals = pd.read_excel(excel_path_2, sheet_name=sheet_name_2)

print(tabulate(df_actuals.head(), headers='keys', tablefmt='psql'))

+----+---------------------+-----------+------------------------------------+------------+-------------+-----------------+------------+-------------+
|    | Date                | Reading   | Location Site Name                 |   Latitude |   Longitude | PV              | Capacity   |   Power(MW) |
|----+---------------------+-----------+------------------------------------+------------+-------------+-----------------+------------+-------------|
|  0 | 2020-02-27 23:00:00 | Actual    | 23MWDistribution PV-83.04 - 40.25  |      40.25 |      -83.05 | Distribution PV | 23MW       |         0   |
|  1 | 2020-09-08 03:00:00 | Actual    | 16MWDistribution PV-83.15 - 41.149 |      41.15 |      -83.15 | Distribution PV | 16MW       |         0   |
|  2 | 2020-09-02 03:00:00 | Actual    | 23MWDistribution PV-84.15 - 39.450 |      39.45 |      -84.15 | Distribution PV | 23MW       |         0   |
|  3 | 2020-10-10 15:00:00 | Actual    | 27MWDistribution PV-84.45 - 39.450 |      39.45 |      -84.

In [7]:
# Check the inferred data types
print(df_actuals.dtypes)

Date                  datetime64[ns]
Reading                       object
Location Site Name            object
Latitude                     float64
Longitude                    float64
PV                            object
Capacity                      object
Power(MW)                    float64
dtype: object


In [8]:
#Let's get some info about the imported dataset.

df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Order ID              599 non-null    int64         
 1   Order Date            599 non-null    datetime64[ns]
 2   Origin Port           599 non-null    object        
 3   Carrier               599 non-null    object        
 4   TPT                   599 non-null    int64         
 5   Service Level         599 non-null    object        
 6   Ship ahead day count  599 non-null    int64         
 7   Ship Late Day count   599 non-null    int64         
 8   Customer              599 non-null    object        
 9   Product ID            599 non-null    int64         
 10  Plant Code            599 non-null    object        
 11  Destination Port      599 non-null    object        
 12  Unit quantity         599 non-null    int64         
 13  Weight              

In [9]:
#Let's get some info about the imported dataset.

df_actuals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Date                499 non-null    datetime64[ns]
 1   Reading             499 non-null    object        
 2   Location Site Name  499 non-null    object        
 3   Latitude            499 non-null    float64       
 4   Longitude           499 non-null    float64       
 5   PV                  499 non-null    object        
 6   Capacity            499 non-null    object        
 7   Power(MW)           499 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 31.3+ KB


In [241]:
# Writing a datframe as a hyper file

'''
The function accept dataframe as input and write it to a hyper file.
'''

def write_via_pantab(df: pd.DataFrame, db_name="Extract_PanTab", schema="Extract", table_name="Extract"):
  # Let's write somewhere besides the default public schema
  table = TableName(schema, table_name)
  pt.frame_to_hyper(df, db_name+".hyper", table=table)

time: 565 µs (started: 2025-06-10 11:42:38 +00:00)


### 📚 Mapping Pandas dtypes → Tableau Hyper SqlType

In [11]:
# Define a mapping from pandas dtypes to Tableau SqlTypes
pandas_to_hyper_type = {
    "int64": SqlType.int(),            # Integer
    "int32": SqlType.int(),
    "float64": SqlType.double(),       # Float
    "float32": SqlType.double(),
    "bool": SqlType.bool(),            # Boolean
    "object": SqlType.text(),          # String/Text
    "string": SqlType.text(),
    "datetime64[ns]": SqlType.timestamp(),  # Datetime
    "timedelta[ns]": SqlType.interval(),    # Optional
    "category": SqlType.text(),        # Fallback to text
    "UInt8": SqlType.int(),            # Unsigned fallback
    "Int8": SqlType.int(),
    "Int16": SqlType.int(),
    "UInt16": SqlType.int(),
    "UInt32": SqlType.int(),
    "UInt64": SqlType.int()
}

### 🛠️ Function to Generate Hyper Schema from DataFrame

In [12]:
def infer_hyper_schema(df, table_name="Extract"):
    column_definitions = []

    for col in df.columns:
        dtype_str = str(df[col].dtype)
        hyper_type = pandas_to_hyper_type.get(dtype_str, SqlType.text())  # default to text
        column=TableDefinition.Column(name=col, type=hyper_type)
        column_definitions.append(column)

    return TableDefinition(table_name=table_name, columns=column_definitions)

## Since pandas treats spatial data as object dtype (e.g., WKT strings), you'll need to:

*   Decide based on column name convention (like geometry, wkt, etc.)
*   Or inspect sample values to detect WKT strings (e.g., "POINT(30 10)", "POLYGON((...))")


### 🧠 Update the Mapping Function

In [13]:
'''
This function will handle spatial columns
User will define which column is including spatial info.

Example
df = pd.DataFrame({
    "location": ["POINT(30 10)", "POINT(40 20)", "POINT(50 30)"],
    "city": ["Cairo", "Dubai", "Riyadh"]
})

schema = infer_hyper_schema_with_spatial(df, spatial_columns=["location"])
print(schema)

'''

def infer_hyper_schema_with_spatial(df, schema="Extract", table_name="Extract", spatial_columns=None):

    spatial_columns = spatial_columns or []

    column_definitions = []

    for col in df.columns:
        if col in spatial_columns:
            hyper_type = SqlType.geography()
        else:
            dtype_str = str(df[col].dtype)
            hyper_type = pandas_to_hyper_type.get(dtype_str, SqlType.text())  # default to text

        column=TableDefinition.Column(name=col, type=hyper_type)
        column_definitions.append(column)


    return TableDefinition(table_name=TableName(schema, table_name), columns=column_definitions)

In [14]:
infer_hyper_schema_with_spatial(df=df_actuals, schema="Extract", table_name="Extract", spatial_columns=None)

TableDefinition(TableName('Extract', 'Extract'), [Column('Date', SqlType.timestamp(), Nullability.NULLABLE), Column('Reading', SqlType.text(), Nullability.NULLABLE), Column('Location Site Name', SqlType.text(), Nullability.NULLABLE), Column('Latitude', SqlType.double(), Nullability.NULLABLE), Column('Longitude', SqlType.double(), Nullability.NULLABLE), Column('PV', SqlType.text(), Nullability.NULLABLE), Column('Capacity', SqlType.text(), Nullability.NULLABLE), Column('Power(MW)', SqlType.double(), Nullability.NULLABLE)], Persistence.PERMANENT)

In [15]:
# Write rows using hyper api

def write_via_hyperapi(df: pd.DataFrame, db_name="Extract_API", schema="Extract", table_name="Extract", spatial_columns=None):
    with HyperProcess(telemetry=Telemetry.DO_NOT_SEND_USAGE_DATA_TO_TABLEAU) as hyper:
        table = infer_hyper_schema_with_spatial(df, schema=schema, table_name=table_name, spatial_columns=spatial_columns)
        with Connection(
            endpoint=hyper.endpoint,
            database=db_name+".hyper",
            create_mode=CreateMode.CREATE_AND_REPLACE,
        ) as conn:
            conn.catalog.create_schema(schema=table.table_name.schema_name)
            # print(table.table_name.schema_name)
            conn.catalog.create_table(table_definition=table)
            with Inserter(conn, table) as inserter:
              inserter.add_rows(rows=df.to_numpy().tolist())
              inserter.execute()

In [257]:
def query_existing_hyper_file(path_to_database, schema, table_name, query):
    """
    Query data from an Hyper file and Create a new table under same schema.
    """
    print("Query data from an Hyper file and Create a new table under same schema")

    # Starts the Hyper Process with telemetry enabled to send data to Tableau.
    # To opt out, simply set telemetry=Telemetry.DO_NOT_SEND_USAGE_DATA_TO_TABLEAU.

    with HyperProcess(telemetry=Telemetry.DO_NOT_SEND_USAGE_DATA_TO_TABLEAU) as hyper:

        # Connect to existing Hyper file.
        with Connection(endpoint=hyper.endpoint,
                        # Using Create Mode Here will raise error
                        database=path_to_database) as connection:

                        # The table names in the "Extract" schema (the default schema).
                        table_names = connection.catalog.get_table_names(schema=schema)
                        table_exists = False

                        for table in table_names:
                          # Confirm that table exists

                          if str(table) == f'"{schema}"."{table_name}"':
                            print(str(table))
                            table_definition = connection.catalog.get_table_definition(name=table)
                            #table_sl = connection.catalog.get_table_definition(name=table)
                            #print(f"Table {table.name} has qualified name: {table}")
                            table_exists = True

                            # get Columns Names for creating a new sliced dataframe based on the provided query
                            col=[]
                            for column in table_definition.columns:
                              #print(f"Column {column.name} has type={column.type} and nullability={column.nullability}")
                              col.append(column.name.unescaped)

                            # Print all rows from the "Extract"."Extract" table.
                            table_name = TableName(schema, table_name)
                            print(f"Sample of quried ros in the table {table_name}: using API approach")
                            # `execute_list_query` executes a SQL query and returns the result as list of rows of data,
                            # each represented by a list of objects.
                            selected_rows = connection.execute_list_query(query=query)

                            df = pd.DataFrame(selected_rows, columns=col)
                            print(tabulate(df.head(), headers='keys', tablefmt='psql'))
                            print('Sliced dataframe has shape:', df.shape)

                            #use timestamp to be added to filename to make it unique
                            #tried to define the creation mode in the connection but failed
                            insertion_ts=f'{datetime.datetime.now():%Y-%m-%d %H:%M:%S}'

                            table_sl_table_name = f"{table_name.name.unescaped}_Sliced_{insertion_ts}"
                            table_sl_db_name=path_to_database.split('/')[-1].split('.')[0]

                            #print(table_definition)
                            #Create a table definition for the sliced data to be inserted in same db, schema
                            table_sl=TableDefinition(table_name=TableName(table_sl_db_name, schema, table_sl_table_name), columns=table_definition.columns)
                            #print(table_sl)

                            connection.catalog.create_table(table_definition=table_sl)
                            with Inserter(connection, table_sl) as inserter:
                              inserter.add_rows(rows=selected_rows)
                              inserter.execute()


                        if not table_exists:
                          print(f"Table {table_name} does not exist")


        print("The connection to the Hyper file has been closed.")
    print("The Hyper process has been shut down.")

time: 1.98 ms (started: 2025-06-10 12:08:28 +00:00)


time: 475 µs (started: 2025-06-10 11:12:34 +00:00)


## Part 2: Method 1 – The Pantab Approach

**Now let’s implement our first method using pantab:**

1.   Set up a path for your Hyper file
2.   Time the process with `time.time()`
3. Use `pantab.frames_to_hyper()` to create a Hyper file with both tables
4. Write SQL queries to filter each table (Weight > 50 for OrderList, Latitude < 40 for Actuals)
5. Use `pantab.frame_from_hyper_query()` to execute each SQL query
6. Save the filtered results back to a Hyper file
7. Calculate and print the processing times

In [250]:
# Use pantab.frames_to_hyper() to create a Hyper file with both tables
write_via_pantab(df=df_orders, db_name="SupplyChain_PanTab", schema="Private", table_name="Orders")
write_via_pantab(df=df_actuals, db_name="SolarEnergy_PanTab", schema="Private", table_name="Actuals")

#Write SQL queries to filter each table (Weight > 50 for OrderList, Latitude < 40 for Actuals)
query_orders = """
SELECT *
FROM "Private"."Orders"
WHERE "Weight" > 50
"""

query_actuals = """
SELECT *
FROM "Private"."Actuals"
WHERE "Latitude" < 40
"""

# Read a subset of the data from the Hyper file
# Use pantab.frame_from_hyper_query() to execute each SQL query
df_orders_filtered = pt.frame_from_hyper_query('/content/SupplyChain_PanTab.hyper', query_orders)
print('Sample records of queried data from SupplyChain dataset using PanTab')
print(tabulate(df_orders_filtered.head(), headers='keys', tablefmt='psql'))
print('Sliced dataframe has shape:', df_orders_filtered.shape)
print('\n')
print('-'*50)

df_actuals_filtered = pt.frame_from_hyper_query('/content/SolarEnergy_PanTab.hyper', query_actuals)
print('Sample records of queried data from SolarEnergy dataset using PanTab')
print(tabulate(df_actuals_filtered.head(), headers='keys', tablefmt='psql'))
print('Sliced dataframe has shape:', df_actuals_filtered.shape)

# Save the filtered results to a Hyper file
# Tried to write the sliced dataframe to same db, schema. However it replaced the whole file.
# This limitatio of pantab
write_via_pantab(df=df_orders_filtered, db_name="SupplyChain_Sliced_PanTab", schema="Private_new", table_name="Orders_sliced")
write_via_pantab(df=df_actuals_filtered, db_name="SolarEnergy_Sliced_PanTab", schema="Private_new", table_name="Actuals_sliced")

Sample records of queried data from SupplyChain dataset using PanTab
+----+------------+---------------------+---------------+-----------+-------+-----------------+------------------------+-----------------------+------------+--------------+--------------+--------------------+-----------------+----------+
|    |   Order ID | Order Date          | Origin Port   | Carrier   |   TPT | Service Level   |   Ship ahead day count |   Ship Late Day count | Customer   |   Product ID | Plant Code   | Destination Port   |   Unit quantity |   Weight |
|----+------------+---------------------+---------------+-----------+-------+-----------------+------------------------+-----------------------+------------+--------------+--------------+--------------------+-----------------+----------|
|  0 | 1447158014 | 2013-05-26 00:00:00 | PORT09        | V44_3     |     1 | CRF             |                      3 |                     0 | V55555_53  |      1700106 | PLANT16      | PORT09             |         

## Part 3: Method 2 – The Hyper API Approach

**Time for the more detailed approach:**

1. Define a path for your second Hyper file
2. Start timing with `time.time()`
3. Use HyperProcess and Connection to create a Hyper database
4. Create appropriate table schemas based on your DataFrame columns
5. Insert data from both DataFrames into your tables
6. Run SQL queries against your tables using the Hyper API
7. Create a new Hyper file with the filtered results
8. Calculate and print the processing times

In [259]:
# Use pantab.frames_to_hyper() to create a Hyper file with both tables

write_via_hyperapi(df=df_orders, db_name="SupplyChain_HyperAPI", schema="Private", table_name="Orders")
write_via_hyperapi(df=df_actuals, db_name="SolarEnergy_HyperAPI", schema="Private", table_name="Actuals")

#Write SQL queries to filter each table (Weight > 50 for OrderList, Latitude < 40 for Actuals)
query_orders = """
SELECT *
FROM "Private"."Orders"
WHERE "Weight" > 50
"""

query_actuals = """
SELECT *
FROM "Private"."Actuals"
WHERE "Latitude" < 40
"""

# Read a subset of the data from the Hyper file
query_existing_hyper_file(path_to_database='/content/SupplyChain_HyperAPI.hyper', schema='Private', table_name='Orders', query=query_orders)
query_existing_hyper_file(path_to_database='/content/SolarEnergy_HyperAPI.hyper', schema='Private', table_name='Actuals', query=query_actuals)

Query data from an Hyper file and Create a new table under same schema
"Private"."Orders"
Sample of quried ros in the table "Private"."Orders": using API approach
+----+------------+---------------------+---------------+-----------+-------+-----------------+------------------------+-----------------------+------------+--------------+--------------+--------------------+-----------------+----------+
|    |   Order ID | Order Date          | Origin Port   | Carrier   |   TPT | Service Level   |   Ship ahead day count |   Ship Late Day count | Customer   |   Product ID | Plant Code   | Destination Port   |   Unit quantity |   Weight |
|----+------------+---------------------+---------------+-----------+-------+-----------------+------------------------+-----------------------+------------+--------------+--------------+--------------------+-----------------+----------|
|  0 | 1447158014 | 2013-05-26 00:00:00 | PORT09        | V44_3     |     1 | CRF             |                      3 |   